# Exoplanet Survey Synthetic Database (SQLite)

This notebook builds a realistic, multi-table SQLite database representing an exoplanet survey. It creates missions, stars, exoplanets, instruments and observation records, and populates them with synthetic but physically inspired data for use in SQL and data science exercises.


In [1]:
# Remove any existing exoplanet.db file so the database is always recreated from a clean state

import os

if os.path.exists("exoplanet.db"):
    os.remove("exoplanet.db")
    print("Old exoplanet.db deleted.")
else:
    print("No existing DB to delete.")


No existing DB to delete.


In [2]:
# Connect to the SQLite database file and enable foreign key enforcement via PRAGMA

import sqlite3

# This will create (or open) a file named exoplanet.db in the Colab environment
db_path = "exoplanet.db"
conn = sqlite3.connect(db_path)

# Create a cursor to execute SQL commands
cur = conn.cursor()

# Always enable foreign key enforcement in SQLite
cur.execute("PRAGMA foreign_keys = ON;")

print("Connected to database:", db_path)


Connected to database: exoplanet.db


In [3]:
# Define and execute the full SQL schema: drop old tables, create all tables, constraints, and foreign keys


schema_sql = """
PRAGMA foreign_keys = ON;

-- Drop tables if they already exist (correct order to avoid FK issues)
DROP TABLE IF EXISTS observations;
DROP TABLE IF EXISTS detection_methods;
DROP TABLE IF EXISTS instruments;
DROP TABLE IF EXISTS exoplanets;
DROP TABLE IF EXISTS stars;
DROP TABLE IF EXISTS missions;

-- 1) Missions table
CREATE TABLE missions (
    mission_id       INTEGER PRIMARY KEY,
    mission_name     TEXT NOT NULL,
    agency           TEXT NOT NULL,
    launch_year      INTEGER NOT NULL CHECK (launch_year >= 1990),
    retirement_year  INTEGER CHECK (retirement_year IS NULL OR retirement_year >= launch_year),
    mission_type     TEXT NOT NULL
);

-- 2) Stars table
CREATE TABLE stars (
    star_id                 INTEGER PRIMARY KEY,
    star_name               TEXT NOT NULL,
    spectral_type           TEXT NOT NULL,
    luminosity_class        TEXT NOT NULL,
    distance_ly             REAL NOT NULL CHECK (distance_ly > 0),
    effective_temperature_k REAL NOT NULL CHECK (effective_temperature_k > 0),
    mass_solar              REAL NOT NULL CHECK (mass_solar > 0),
    metallicity_fe_h        REAL,
    discovery_mission_id    INTEGER,
    FOREIGN KEY (discovery_mission_id) REFERENCES missions(mission_id)
);

-- 3) Exoplanets table
CREATE TABLE exoplanets (
    planet_id                 INTEGER PRIMARY KEY,
    star_id                   INTEGER NOT NULL,
    planet_name               TEXT NOT NULL,
    planet_type               TEXT NOT NULL,
    orbital_period_days       REAL NOT NULL CHECK (orbital_period_days > 0),
    semi_major_axis_au        REAL NOT NULL CHECK (semi_major_axis_au > 0),
    planet_radius_earth       REAL NOT NULL CHECK (planet_radius_earth > 0),
    planet_mass_earth         REAL NOT NULL CHECK (planet_mass_earth > 0),
    equilibrium_temperature_k REAL NOT NULL CHECK (equilibrium_temperature_k > 0),
    discovery_year            INTEGER NOT NULL CHECK (discovery_year BETWEEN 1990 AND 2025),
    habitability_class        INTEGER CHECK (habitability_class BETWEEN 1 AND 3),
    FOREIGN KEY (star_id) REFERENCES stars(star_id)
);

-- 4) Instruments table
CREATE TABLE instruments (
    instrument_id    INTEGER PRIMARY KEY,
    mission_id       INTEGER NOT NULL,
    instrument_name  TEXT NOT NULL,
    wavelength_band  TEXT NOT NULL,
    resolution_level INTEGER NOT NULL CHECK (resolution_level BETWEEN 1 AND 3),
    FOREIGN KEY (mission_id) REFERENCES missions(mission_id)
);

-- 5) Detection methods table
CREATE TABLE detection_methods (
    method_id       INTEGER PRIMARY KEY,
    method_name     TEXT NOT NULL,
    method_category TEXT NOT NULL
);

-- 6) Observations table (main, with composite primary key)
CREATE TABLE observations (
    planet_id            INTEGER NOT NULL,
    instrument_id        INTEGER NOT NULL,
    obs_timestamp        TEXT NOT NULL,  -- store as ISO 8601 string (e.g., '2022-03-15T22:15:00')
    star_id              INTEGER NOT NULL,
    method_id            INTEGER NOT NULL,
    signal_to_noise      REAL NOT NULL CHECK (signal_to_noise >= 0),
    detection_confidence INTEGER NOT NULL CHECK (detection_confidence BETWEEN 1 AND 4),
    transit_depth_ppm    REAL,
    radial_velocity_ms   REAL,
    is_primary_detection INTEGER NOT NULL CHECK (is_primary_detection IN (0,1)),
    quality_flag         INTEGER NOT NULL CHECK (quality_flag BETWEEN 1 AND 4),
    PRIMARY KEY (planet_id, instrument_id, obs_timestamp),
    FOREIGN KEY (planet_id)     REFERENCES exoplanets(planet_id),
    FOREIGN KEY (instrument_id) REFERENCES instruments(instrument_id),
    FOREIGN KEY (star_id)       REFERENCES stars(star_id),
    FOREIGN KEY (method_id)     REFERENCES detection_methods(method_id)
);
"""

cur.executescript(schema_sql)
conn.commit()

print("All tables created successfully.")


All tables created successfully.


In [4]:
# List all tables currently defined in the SQLite database to verify successful schema creation

cur.execute("SELECT name FROM sqlite_master WHERE type='table' ORDER BY name;")
tables = cur.fetchall()
tables


[('detection_methods',),
 ('exoplanets',),
 ('instruments',),
 ('missions',),
 ('observations',),
 ('stars',)]

In [5]:
# Display the CREATE TABLE SQL definition for the observations table to inspect its structure

cur.execute("""
    SELECT sql
    FROM sqlite_master
    WHERE type = 'table' AND name = 'observations';
""")
print(cur.fetchone()[0])


CREATE TABLE observations (
    planet_id            INTEGER NOT NULL,
    instrument_id        INTEGER NOT NULL,
    obs_timestamp        TEXT NOT NULL,  -- store as ISO 8601 string (e.g., '2022-03-15T22:15:00')
    star_id              INTEGER NOT NULL,
    method_id            INTEGER NOT NULL,
    signal_to_noise      REAL NOT NULL CHECK (signal_to_noise >= 0),
    detection_confidence INTEGER NOT NULL CHECK (detection_confidence BETWEEN 1 AND 4),
    transit_depth_ppm    REAL,
    radial_velocity_ms   REAL,
    is_primary_detection INTEGER NOT NULL CHECK (is_primary_detection IN (0,1)),
    quality_flag         INTEGER NOT NULL CHECK (quality_flag BETWEEN 1 AND 4),
    PRIMARY KEY (planet_id, instrument_id, obs_timestamp),
    FOREIGN KEY (planet_id)     REFERENCES exoplanets(planet_id),
    FOREIGN KEY (instrument_id) REFERENCES instruments(instrument_id),
    FOREIGN KEY (star_id)       REFERENCES stars(star_id),
    FOREIGN KEY (method_id)     REFERENCES detection_methods(metho

In [6]:
# Inspect the schema details for the observations table (column names, types, and constraints)
cur.execute("PRAGMA table_info(observations);")
for col in cur.fetchall():
    print(col)


(0, 'planet_id', 'INTEGER', 1, None, 1)
(1, 'instrument_id', 'INTEGER', 1, None, 2)
(2, 'obs_timestamp', 'TEXT', 1, None, 3)
(3, 'star_id', 'INTEGER', 1, None, 0)
(4, 'method_id', 'INTEGER', 1, None, 0)
(5, 'signal_to_noise', 'REAL', 1, None, 0)
(6, 'detection_confidence', 'INTEGER', 1, None, 0)
(7, 'transit_depth_ppm', 'REAL', 0, None, 0)
(8, 'radial_velocity_ms', 'REAL', 0, None, 0)
(9, 'is_primary_detection', 'INTEGER', 1, None, 0)
(10, 'quality_flag', 'INTEGER', 1, None, 0)


In [7]:
# (Optional) Helper cell to download the SQLite database from Colab to your local machine (kept commented)
# from google.colab import files

# files.download("exoplanet.db")


In [8]:


# Import helper libraries and reconnect to the database so we can generate realistic random data
import random
import math
from datetime import datetime, timedelta

db_path = "exoplanet.db"
conn = sqlite3.connect(db_path)
cur = conn.cursor()

# Always turn on foreign keys
cur.execute("PRAGMA foreign_keys = ON;")

print("Reconnected to:", db_path)


Reconnected to: exoplanet.db


In [9]:
# Insert a small catalogue of space missions into the missions table (used as a lookup in other tables)
missions_data = [
    (1, "Kepler",        "NASA", "Space Telescope", 2009, 2018),
    (2, "TESS",          "NASA", "Space Telescope", 2018, None),
    (3, "CHEOPS",        "ESA",  "Space Telescope", 2019, None),
    (4, "Hubble",        "NASA", "Space Telescope", 1990, 2021),
    (5, "James Webb",    "NASA/ESA/CSA", "Space Telescope", 2021, None),
]

cur.executemany("""
    INSERT INTO missions (mission_id, mission_name, agency, mission_type, launch_year, retirement_year)
    VALUES (?, ?, ?, ?, ?, ?)
""", missions_data)

conn.commit()
print("Inserted missions:", len(missions_data))


Inserted missions: 5


In [10]:
# Insert a set of common exoplanet detection methods into the detection_methods table
detection_methods_data = [
    (1, "Transit",         "Photometric"),
    (2, "Radial Velocity", "Spectroscopic"),
    (3, "Direct Imaging",  "Imaging"),
    (4, "Microlensing",    "Gravitational"),
]

cur.executemany("""
    INSERT INTO detection_methods (method_id, method_name, method_category)
    VALUES (?, ?, ?)
""", detection_methods_data)

conn.commit()
print("Inserted detection methods:", len(detection_methods_data))


Inserted detection methods: 4


In [11]:
# Insert instruments for each mission, including wavelength band and an ordinal resolution level
instruments_data = [
    (1, 1, "Kepler Photometer",     "Optical", 3),
    (2, 2, "TESS Camera",           "Optical", 2),
    (3, 3, "CHEOPS Imager",         "Optical", 2),
    (4, 4, "Hubble WFC3",           "Optical", 3),
    (5, 5, "JWST NIRCam",           "Infrared", 3),
    (6, 5, "JWST NIRSpec",          "Infrared", 3),
]

cur.executemany("""
    INSERT INTO instruments (instrument_id, mission_id, instrument_name, wavelength_band, resolution_level)
    VALUES (?, ?, ?, ?, ?)
""", instruments_data)

conn.commit()
print("Inserted instruments:", len(instruments_data))


Inserted instruments: 6


In [12]:
# Define spectral type categories and a helper function to generate semi-random (sometimes duplicated) star names
spectral_types = ["F5V", "G2V", "K1V", "K5V", "M1V", "M3V"]
luminosity_classes = ["V", "IV", "III"]  # Dwarf, subgiant, giant

def random_star_name(index):
    # e.g., "Star-101", "Star-102" plus a few repeats to simulate duplicates
    base = 100 + (index % 150)
    return f"Star-{base}"


In [13]:
# Generate synthetic stars with realistic properties and occasional missing metallicity values
num_stars = 200
stars_data = []

for i in range(1, num_stars + 1):
    star_id = i
    star_name = random_star_name(i)  # intentionally will create some name repeats
    spectral_type = random.choice(spectral_types)
    luminosity_class = random.choice(luminosity_classes)

    # Distance between 10 and 5000 light years
    distance_ly = round(random.uniform(10, 5000), 2)

    # Temperature depends loosely on spectral type
    if spectral_type.startswith("F"):
        temp = random.uniform(6000, 7500)
    elif spectral_type.startswith("G"):
        temp = random.uniform(5200, 6000)
    elif spectral_type.startswith("K"):
        temp = random.uniform(3900, 5200)
    else:  # M
        temp = random.uniform(2400, 3900)
    effective_temperature_k = round(temp, 1)

    # Mass (solar) roughly based on type
    if spectral_type.startswith("F"):
        mass = random.uniform(1.1, 1.5)
    elif spectral_type.startswith("G"):
        mass = random.uniform(0.9, 1.1)
    elif spectral_type.startswith("K"):
        mass = random.uniform(0.6, 0.9)
    else:  # M
        mass = random.uniform(0.1, 0.6)
    mass_solar = round(mass, 3)

    # Metallicity: some missing, some between -0.5 and +0.5
    if random.random() < 0.2:  # 20% NULL metallicity
        metallicity = None
    else:
        metallicity = round(random.uniform(-0.5, 0.5), 3)

    # Some stars discovered by a mission, some not
    discovery_mission_id = random.choice([1, 2, 3, 4, 5, None])

    stars_data.append((
        star_id, star_name, spectral_type, luminosity_class,
        distance_ly, effective_temperature_k, mass_solar,
        metallicity, discovery_mission_id
    ))

cur.executemany("""
    INSERT INTO stars (
        star_id, star_name, spectral_type, luminosity_class,
        distance_ly, effective_temperature_k, mass_solar,
        metallicity_fe_h, discovery_mission_id
    ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
""", stars_data)

conn.commit()
print("Inserted stars:", len(stars_data))


Inserted stars: 200


In [14]:
# Define the high-level exoplanet type categories used when generating planets
planet_types = ["Hot Jupiter", "Warm Neptune", "Super Earth", "Earth-like", "Mini Neptune"]


In [15]:
# Generate synthetic exoplanets linked to stars, with physics-inspired relationships between mass, radius and period
num_planets = 300
exoplanets_data = []

for planet_id in range(1, num_planets + 1):
    # Random star; some stars get 0 planets, some multiple
    star_id = random.randint(1, num_stars)

    # Planet name: star_name + letter
    letter = chr(ord('b') + (planet_id % 5))  # b, c, d, e, f cycle
    planet_name = f"{stars_data[star_id - 1][1]}-{letter}"  # uses star_name from stars_data
    planet_type = random.choice(planet_types)

    # Orbital period (days) & semi-major axis (AU)
    if planet_type == "Hot Jupiter":
        orbital_period_days = random.uniform(1, 10)
        semi_major_axis_au = random.uniform(0.01, 0.1)
        planet_radius_earth = random.uniform(8, 15)
        planet_mass_earth = random.uniform(50, 500)
    elif planet_type == "Warm Neptune":
        orbital_period_days = random.uniform(10, 100)
        semi_major_axis_au = random.uniform(0.1, 0.5)
        planet_radius_earth = random.uniform(3, 7)
        planet_mass_earth = random.uniform(10, 50)
    elif planet_type == "Super Earth":
        orbital_period_days = random.uniform(10, 200)
        semi_major_axis_au = random.uniform(0.1, 1.0)
        planet_radius_earth = random.uniform(1.5, 3)
        planet_mass_earth = random.uniform(5, 15)
    elif planet_type == "Earth-like":
        orbital_period_days = random.uniform(200, 500)
        semi_major_axis_au = random.uniform(0.7, 1.5)
        planet_radius_earth = random.uniform(0.8, 1.5)
        planet_mass_earth = random.uniform(0.5, 5)
    else:  # Mini Neptune
        orbital_period_days = random.uniform(20, 300)
        semi_major_axis_au = random.uniform(0.2, 1.5)
        planet_radius_earth = random.uniform(2, 5)
        planet_mass_earth = random.uniform(5, 20)

    orbital_period_days = round(orbital_period_days, 2)
    semi_major_axis_au = round(semi_major_axis_au, 3)
    planet_radius_earth = round(planet_radius_earth, 2)
    planet_mass_earth = round(planet_mass_earth, 2)

    # Equilibrium temperature (approx): closer = hotter
    equilibrium_temp = random.uniform(300, 2500)
    equilibrium_temperature_k = round(equilibrium_temp, 1)

    # Discovery year
    discovery_year = random.randint(1990, 2025)

    # Habitability: some planets not evaluated (NULL)
    if planet_type == "Earth-like" and random.random() < 0.7:
        habitability_class = random.choice([2, 3])
    elif planet_type == "Super Earth" and random.random() < 0.4:
        habitability_class = random.choice([2, 3])
    else:
        # sometimes None (no classification), sometimes clearly non-habitable
        habitability_class = None if random.random() < 0.3 else 1

    exoplanets_data.append((
        planet_id, star_id, planet_name, planet_type,
        orbital_period_days, semi_major_axis_au,
        planet_radius_earth, planet_mass_earth,
        equilibrium_temperature_k, discovery_year,
        habitability_class
    ))

cur.executemany("""
    INSERT INTO exoplanets (
        planet_id, star_id, planet_name, planet_type,
        orbital_period_days, semi_major_axis_au,
        planet_radius_earth, planet_mass_earth,
        equilibrium_temperature_k, discovery_year,
        habitability_class
    ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
""", exoplanets_data)

conn.commit()
print("Inserted exoplanets:", len(exoplanets_data))


Inserted exoplanets: 300


In [16]:
# Define a helper to draw random observation timestamps and prepare to build observation records for each planet
def random_timestamp(start_year=2010, end_year=2025):
    start = datetime(start_year, 1, 1)
    end = datetime(end_year, 12, 31)
    delta = end - start
    random_days = random.randint(0, delta.days)
    random_seconds = random.randint(0, 24*3600 - 1)
    dt = start + timedelta(days=random_days, seconds=random_seconds)
    # ISO 8601 string
    return dt.strftime("%Y-%m-%dT%H:%M:%S")


In [17]:
# Generate detailed observation records per planet (instrument, method, SNR, confidence, transit/RV, quality flags)
observations_data = []

# Convenience: list of instrument_ids and method_ids
instrument_ids = [row[0] for row in instruments_data]         # first element in each tuple was instrument_id
method_ids = [row[0] for row in detection_methods_data]       # method_id from detection_methods

for planet in exoplanets_data:
    planet_id = planet[0]
    star_id = planet[1]

    # Each planet gets between 3 and 10 observations
    num_obs_for_planet = random.randint(3, 10)

    for _ in range(num_obs_for_planet):
        instrument_id = random.choice(instrument_ids)
        method_id = random.choices(
            population=method_ids,
            weights=[0.6, 0.25, 0.1, 0.05],  # Transit more common
            k=1
        )[0]

        obs_timestamp = random_timestamp()

        # Signal-to-noise ratio
        if method_id == 1:  # Transit
            snr = random.uniform(10, 500)
        elif method_id == 2:  # Radial Velocity
            snr = random.uniform(5, 300)
        else:
            snr = random.uniform(1, 100)
        signal_to_noise = round(snr, 2)

        # Detection confidence (ordinal 1-4)
        # Higher SNR -> likely higher confidence
        if signal_to_noise > 200:
            detection_confidence = random.choice([3, 4])
        elif signal_to_noise > 50:
            detection_confidence = random.choice([2, 3])
        else:
            detection_confidence = random.choice([1, 2])

        # Quality flag (1-4)
        quality_flag = random.choices([1, 2, 3, 4], weights=[0.1, 0.2, 0.4, 0.3], k=1)[0]

        # Transit depth / radial velocity: NULL when not applicable
        if method_id == 1:  # Transit
            transit_depth_ppm = round(random.uniform(50, 20000), 1)
            # small chance of missing value even when transit
            if random.random() < 0.05:
                transit_depth_ppm = None
            radial_velocity_ms = None
        elif method_id == 2:  # Radial Velocity
            transit_depth_ppm = None
            rv = random.uniform(0.1, 500.0)
            radial_velocity_ms = round(rv, 2)
            if random.random() < 0.05:
                radial_velocity_ms = None
        else:
            # other methods: both often NULL
            transit_depth_ppm = None
            radial_velocity_ms = None

        # primary detection roughly first 1-2 observations per planet
        is_primary_detection = 1 if random.random() < 0.2 else 0

        observations_data.append((
            planet_id, instrument_id, obs_timestamp,
            star_id, method_id,
            signal_to_noise, detection_confidence,
            transit_depth_ppm, radial_velocity_ms,
            is_primary_detection, quality_flag
        ))

len(observations_data)


1901

In [18]:
# Bulk insert all generated observation rows into the observations table and commit the transaction
cur.executemany("""
    INSERT INTO observations (
        planet_id, instrument_id, obs_timestamp,
        star_id, method_id,
        signal_to_noise, detection_confidence,
        transit_depth_ppm, radial_velocity_ms,
        is_primary_detection, quality_flag
    ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
""", observations_data)

conn.commit()
print("Inserted observations:", len(observations_data))


Inserted observations: 1901


In [19]:
# Quick sanity check: count how many rows are in each table in the database
for table in ["missions", "stars", "exoplanets", "instruments", "detection_methods", "observations"]:
    cur.execute(f"SELECT COUNT(*) FROM {table}")
    count = cur.fetchone()[0]
    print(f"{table}: {count} rows")


missions: 5 rows
stars: 200 rows
exoplanets: 300 rows
instruments: 6 rows
detection_methods: 4 rows
observations: 1901 rows


In [20]:
# Count how many stars are missing metallicity values to illustrate deliberate missing data
cur.execute("SELECT COUNT(*) FROM stars WHERE metallicity_fe_h IS NULL;")
print("Stars with NULL metallicity:", cur.fetchone()[0])


Stars with NULL metallicity: 45


In [21]:
# Count how many observations are missing transit depth to show method-dependent missingness
cur.execute("SELECT COUNT(*) FROM observations WHERE transit_depth_ppm IS NULL;")
print("Observations with NULL transit_depth_ppm:", cur.fetchone()[0])


Observations with NULL transit_depth_ppm: 854


In [22]:
# Count how many observations are missing radial velocity to complement the transit depth check
cur.execute("SELECT COUNT(*) FROM observations WHERE radial_velocity_ms IS NULL;")
print("Observations with NULL radial_velocity_ms:", cur.fetchone()[0])


Observations with NULL radial_velocity_ms: 1417


In [23]:
# List star names that appear more than once, illustrating deliberate duplicate categorical values
cur.execute("""
    SELECT star_name, COUNT(*)
    FROM stars
    GROUP BY star_name
    HAVING COUNT(*) > 1
    LIMIT 10;
""")
print("Star names with duplicates:")
for row in cur.fetchall():
    print(row)


Star names with duplicates:
('Star-101', 2)
('Star-102', 2)
('Star-103', 2)
('Star-104', 2)
('Star-105', 2)
('Star-106', 2)
('Star-107', 2)
('Star-108', 2)
('Star-109', 2)
('Star-110', 2)


In [24]:
# Download the finished exoplanet.db file from Colab to your local machine
from google.colab import files
files.download("exoplanet.db")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>